# **Taller 1: Conceptos Básicos de Teledetección**

**Asignatura:** Teledetección

**Presentado por:** Laura Alejandra Díaz López

**Código:** 202522993

## **Actividad 1: Exploración de Imágenes Satelitales en Google Earth Engine**

**_Objetivo:_** Familiarizarse con la visualización de imágenes satelitales y la diferencia entre sensores pasivos y activos.

En esta actividad se exploraron imágenes satelitales para la ciudad de Manizales, Caldas, Colombia, utilizando Google Earth Engine (GEE). Para lo anterior se utilizó el siguiente script de JavaScript:

---

``` JavaScript
// Definir la región de interés (Manizales, Caldas, Colombia)
var manizales = ee.Geometry.Point([-75.50911, 5.06379]);

// Cargar imágenes de Landsat 8 (sensor pasivo) y Sentinel-1 (sensor activo)
var landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
                .filterBounds(manizales)
                .filterDate('2024-01-01', '2024-12-31')
                .median();

var sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")
                  .filterBounds(manizales) 
                  .filterDate('2024-01-01', '2024-12-31')
                  .median();

// Visualización
Map.centerObject(manizales, 10);
Map.addLayer(landsat, {bands: ['B4', 'B3', 'B2'], min: 0, max: 0.3}, 'Landsat 8 (Óptico)');
Map.addLayer(sentinel1, {min: -20, max: 0}, 'Sentinel-1 (Radar)');
```

---

El anterior script permite definir las coordenadas de la zona de interés, en este caso Manizales, para luego cargar una imagen óptica de Landsat 8 (entre el 01/01/2024 y el 31/12/2024) y una de radar Sentinel-1 (entre el 01/01/2024 y el 31/12/2024).

A continuación, se muestra el mapa base para la zona de interés (con las capas desactivadas):

<div style="text-align: center;">
<img src="./images/act1_img1.png" alt="Localización Manizales" width="1000"/>
</div>

En la siguiente captura se observa la imagen de radar Sentinel-1 para la zona, en esta se logran detectar estructuras y relieve de forma clara.

<div style="text-align: center;">
<img src="./images/act1_img2.png" alt="Imagen satelital de radar Sentinel-1" width="1000"/>
</div>

En la siguiente captura se puede visualizar la imagen óptica Landsat 8 para Manizales, con colores naturales, lo que facilita la interpretación de la cobertura, pero que está bastante afectada por la nubosidad.

<div style="text-align: center;">
<img src="./images/act1_img3.png" alt="Imagen satelital óptica Landsat 8" width="1000"/>
</div>

**¿Cuáles son las ventajas de cada tipo de sensor en términos de nubosidad y detección de estructuras en la superficie?**

Los sensores pasivos como Landsat 8 capturan imágenes en luz visible, ideales para observar vegetación y cuerpos de agua como las verían nuestros ojos, pero pueden verse afectadas por la presencia de nubes que bloquean la radiación. En cambio, los sensores activos como Sentinel-1 emiten microondas que logran penetrar las nubes y permiten captar estructuras y relieve incluso bajo condiciones de alta nubosidad. Se puede combinar el uso de ambos tipos de imágenes satelitales para complementarse.

## **Actividad 2: Descarga y Visualización de Imágenes Satelitales en QGIS**

**_Objetivo:_** Descargar imágenes satelitales gratuitas y analizarlas en un software SIG.

Para esta actividad se realizó la descarga de una imagen satelital Landsat 8 para Manizales, Caldas, Colombia utilizando Google Earth Engine (GEE) por medio del sigueinte código de JavaScript:

---

```JavaScript
// Definir la región de interés (Manizales, Caldas, Colombia)
var manizales = ee.Geometry.Point([-75.50911, 5.06379]);

// Filtrar la colección de imágenes Landsat 8 SR (sin filtrar por nubosidad aún)
var landsatCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
                          .filterBounds(manizales) 
                          .filterDate('2024-01-01', '2024-12-31'); // Se amplía el rango de fechas

// Contar imágenes disponibles
var count = landsatCollection.size();
print("Número de imágenes disponibles:", count);

// Mostrar lista de imágenes con fechas y cobertura de nubes
var imageList = landsatCollection.map(function(image) {
  return ee.Feature(null, {
    'ID': image.id(),
    'Fecha': image.date().format('YYYY-MM-dd'),
    'Nubosidad': image.get('CLOUD_COVER')
  });
});

print("Lista de imágenes encontradas:", imageList);

// Si hay imágenes disponibles, procesamos la mejor
if (count.gt(0)) {
  var bestImage = landsatCollection.sort('CLOUD_COVER').first(); // Tomamos la de menor nubosidad
  print("Imagen seleccionada:", bestImage);

  // Seleccionar bandas de reflectancia superficial
  var bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']; 
  var scaledImage = bestImage.select(bands).multiply(0.0000275).add(-0.2); 

  // Visualizar en el mapa
  var visParams = {bands: ['SR_B4', 'SR_B3', 'SR_B2'], min: 0, max: 0.3, gamma: 1.4};
  Map.centerObject(manizales, 10);
  Map.addLayer(scaledImage, visParams, "Landsat 8 SR - Manizales");

  // Exportar la imagen a Google Drive
  Export.image.toDrive({
    image: scaledImage,
    description: "Landsat8_SR_Manizales",
    folder: "GEE_Exports",
    fileNamePrefix: "Landsat8_SR_Manizales",
    region: manizales.buffer(5000).bounds(),
    scale: 30,
    maxPixels: 1e13,
    fileFormat: "GeoTIFF"
  });

} else {
  print("❌ No se encontraron imágenes en el período seleccionado.");
}
```

---

A continuación, se muestra el resultado de ejecutar el anterior código en Google Earth Engine:

<div style="text-align: center;">
<img src="./images/act2_img1.png" alt="Resultado en Google Earth Engine (GEE)" width="1000"/>
</div>

En la pestaña **`Tasks`** se ejecutó la exportación. En la siguiente imagen se muestra la configuración para la exportación:

<div style="text-align: center;">
<img src="./images/act2_img2.png" alt="Exportación de la imagen" width="1000"/>
</div>

En la siguiente captura se logra verificar la correcta exportación a la carpeta `GEE_Exports` en Google Drive, donde es posible descargarla.

<div style="text-align: center;">
<img src="./images/act2_img3.png" alt="Imagen exportada a Google Drive" width="1000"/>
</div>

A continuación, se muestra información básica de la imagen:

In [8]:
import rasterio

def print_raster_info(path):
    with rasterio.open(path) as dataset:
        print(f"Archivo: {dataset.name}")
        print(f"Número de bandas: {dataset.count}")
        print(f"CRS: {dataset.crs}")
        print(f"Formato: {dataset.driver}")
        print(f"Tipo de datos por píxel: {dataset.dtypes[0]}")  # dtype de la primera banda
        bits = int(dataset.dtypes[0].replace('uint', '').replace('int', '').replace('float', ''))
        print(f"Bits por pixel: {bits}")
        print(f"Resolución (pixel size): {dataset.res}")

tif_path = r'GEE_Exports\Landsat8_SR_Manizales.tif' # Ruta del archivo TIFF
print_raster_info(tif_path)

Archivo: GEE_Exports\Landsat8_SR_Manizales.tif
Número de bandas: 6
CRS: EPSG:32618
Formato: GTiff
Tipo de datos por píxel: float64
Bits por pixel: 64
Resolución (pixel size): (30.0, 30.0)


**Visualización en QGIS**

Luego de descargar la imagen desde Google Drive, se abrieron QGIS y SNAP para crear composiciones en falso color de la imagen.

- **Composición para vegetación en rojo (NDVI):** Bandas B5, B4 y B3. Esta combinación resalta la vegetación en tonos rojos a través del índice NDVI.
 
  - **En QGIS:**
  
    <div style="text-align: center;">
    <img src="./images/act2_img4_0.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>

  - **En SNAP:**
  
    <div style="text-align: center;">
    <img src="./images/act2_img4_1.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>

- **Composición en falso color infrarrojo térmico:** Bandas B6, B5 y B4. Permite observar diferencias térmicas y propiedades superficiales.
 
  - **En QGIS:**
  
    <div style="text-align: center;">
    <img src="./images/act2_img5_0.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>

  - **En SNAP:**
  
    <div style="text-align: center;">
    <img src="./images/act2_img5_1.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>

**¿Cómo cambia la percepción de los objetos en función de la combinación de bandas?**

La percepción visual de los objetos cambia notablemente según la combinación de bandas usada pues cada combinación modifica los colores visibles. En la composición para vegetación en rojo (B5, B4, B3), la vegetación aparece en rojo brillante, facilitando la identificación de zonas con vegetación y densidad de esta. En cambio, la composición falso color infrarrojo térmico (B6, B5, B4) destaca características térmicas y diferencias en humedad en el suelo o la vegetación.


## **Actividad 3: Diferencia entre Sensores Activos y Pasivos**

**_Objetivo:_** Comparar imágenes ópticas y de radar en una misma región.

Se descargaron imágenes Sentinel-2 (ópticas) y Sentinel-1 (radar) para la región de Manizales, por medio de Google Earth Engine, ejecutando el siguiente código:

---

``` JavaScript
// 📍 Definir la región de interés (Manizales, Caldas, Colombia)
var manizales = ee.Geometry.Point([-75.50911, 5.06379]);

// 🔹 FILTRAR IMAGEN SENTINEL-2 (Óptico) - Nivel 2A con corrección atmosférica
var sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR")
                  .filterBounds(manizales)
                  .filterDate('2024-01-01', '2024-12-31')
                  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
                  .sort('CLOUDY_PIXEL_PERCENTAGE')
                  .first(); // Seleccionar la imagen con menos nubes

print("Imagen Sentinel-2 seleccionada:", sentinel2);
print("Bandas Sentinel-2:", sentinel2.bandNames());

// 🔹 FILTRAR IMAGEN SENTINEL-1 (Radar) - Polarización VV y VH
var sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")
                  .filterBounds(manizales)
                  .filterDate('2025-01-01', '2025-12-31')
                  .filterMetadata('instrumentMode', 'equals', 'IW')
                  .filterMetadata('orbitProperties_pass', 'equals', 'DESCENDING')
                  .filterMetadata('resolution_meters', 'equals', 10)
                  .first(); // Tomar la primera imagen disponible

print("Imagen Sentinel-1 seleccionada:", sentinel1);
print("Bandas Sentinel-1:", sentinel1.bandNames());

// 📌 VISUALIZACIÓN
// Sentinel-2 (Óptico) - RGB Natural
var visS2 = {bands: ['B4', 'B3', 'B2'], min: 0, max: 3000, gamma: 1.4};
Map.centerObject(manizales, 10);
Map.addLayer(sentinel2, visS2, "Sentinel-2 (Óptico)");

// Sentinel-1 (Radar) - Polarización VV
var visS1 = {bands: ['VV'], min: -25, max: 0, gamma: 1};
Map.addLayer(sentinel1, visS1, "Sentinel-1 (Radar VV)");

// 📤 EXPORTAR SENTINEL-2 A GOOGLE DRIVE
Export.image.toDrive({
  image: sentinel2.select(['B4', 'B3', 'B2', 'B8']), // Rojo, Verde, Azul, Infrarrojo Cercano
  description: "Sentinel2_Manizales",
  folder: "GEE_Exports",
  fileNamePrefix: "Sentinel2_Manizales",
  region: manizales.buffer(5000).bounds(),
  scale: 10,
  maxPixels: 1e13,
  fileFormat: "GeoTIFF"
});

// 📤 EXPORTAR SENTINEL-1 A GOOGLE DRIVE
Export.image.toDrive({
  image: sentinel1.select(['VV', 'VH']), // Polarizaciones VV y VH
  description: "Sentinel1_Manizales",
  folder: "GEE_Exports",
  fileNamePrefix: "Sentinel1_Manizales",
  region: manizales.buffer(5000).bounds(),
  scale: 10,
  maxPixels: 1e13,
  fileFormat: "GeoTIFF"
});
```

---

El código anterior generó el siguiente resultado en Google Earth Engine, donde se pueden ver las capas "Sentinel-1 (Radar VV)" y "Sentinel-2 (Óptico)":

<div style="text-align: center;">
<img src="./images/act3_img1.png" alt="Resultado en Google Earth Engine (GEE)" width="1000"/>
</div>

En la pestaña **`Tasks`** se ejecutó la exportación para ambas imágenes. En la siguiente imagen se muestra la configuración para su exportación:

<div style="text-align: center;">
<img src="./images/act3_img2.png" alt="Exportación de imágenes Sentinel-1 y Sentinel-2" width="750"/>
</div>

Se verificó la correcta exportación de las imágenes a la carpeta `GEE_Exports` en Google Drive, donde se encontraba la imagen de Landsat 8 exportada en el punto anterior.

<div style="text-align: center;">
<img src="./images/act3_img3.png" alt="Imágenes exportadas a Google Drive" width="1000"/>
</div>

A continuación, se muestra información básica de la imagen Sentinel-1:

In [6]:
tif_path = r'GEE_Exports\Sentinel1_Manizales.tif'
print_raster_info(tif_path)

Archivo: GEE_Exports\Sentinel1_Manizales.tif
Número de bandas: 2
CRS: EPSG:32618
Formato: GTiff
Tipo de datos por píxel: float64
Bits por pixel: 64
Resolución (pixel size): (10.0, 10.0)


Y la información básica para la imagen Sentinel-2:

In [7]:
tif_path = r'GEE_Exports\Sentinel2_Manizales.tif'
print_raster_info(tif_path)

Archivo: GEE_Exports\Sentinel2_Manizales.tif
Número de bandas: 4
CRS: EPSG:32618
Formato: GTiff
Tipo de datos por píxel: uint16
Bits por pixel: 16
Resolución (pixel size): (10.0, 10.0)



**Visualización en QGIS y SNAP**

Luego de descargadas las imagenes desde Google Drive, se procedió a cargarlas en los softwares QGIS y SNAP. A continuación, se observa cada una de las imágenes.

- **Sentinel-1 (sensor radar):** Permite apreciar la textura y estructura del terreno, independientemente de las condiciones atmosféricas y la iluminación.
  
  - **En QGIS:**
  
    <div style="text-align: center;">
    <img src="./images/act3_img4_0.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>

  - **En SNAP:**
  
    <div style="text-align: center;">
    <img src="./images/act3_img4_1.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>

- **Sentinel-2 (sensor óptico):** Permite observar cuerpos de agua, vegetación y zonas urbanas con alta resolución y colores naturales.
  
  - **En QGIS:**
  
    <div style="text-align: center;">
    <img src="./images/act3_img5_0.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>

  - **En SNAP:**
  
    <div style="text-align: center;">
    <img src="./images/act3_img5_1.png" alt="Imagen Sentinel-1" width="1000"/>
    </div>


**¿En qué condiciones sería más útil usar imágenes de radar en lugar de ópticas?** 

Es más útil usar imágenes de radar en condiciones de alta nubosidad, oscuridad o lluvias, donde las imágenes ópticas tienen limitaciones. Además de casos en los que es importante estudiar la estructura de la superficie.